In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from joblib import Parallel, delayed
from tqdm import tqdm
from itertools import product
from itertools import permutations
from itertools import combinations
from pyEDM import *
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
import time
import os
import math
import random
from scipy.stats import ttest_ind
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))

import warnings
warnings.filterwarnings("ignore", 
    message="A worker stopped while some jobs were given to the executor.",
    module="joblib.externals.loky.process_executor")

In [2]:
def get_block(data, num_lags=1, tau=1):
    ''' Get a dataframe with all the possible valid lags of the variables. '''
    
    block = pd.concat([data[var].shift(lag*tau).rename(f'{var}(t-{lag*tau})') for lag in range(num_lags+1) for var in data.columns], axis=1)

    return block

In [3]:
def ccm(interaction, block, E_list, tau_list, theta_list, Tp, sample=50, sig=0.05):
    #solver = HistGradientBoostingRegressor() #TRYING DIFFERNT SOLVER TO ENSURE CONVERGENCE



    print(interaction)
    lib = f'1 {len(block)}'
    
    # Get dataframe with two species of interest
    A = interaction[0]; B = interaction[1]
    df = block[[f'{A}(t-0)', f'{B}(t-0)']]
    
    driver = f'{A}(t-0)'    
    default_output = {
        'target (driver)': A,
        'lib (driven)': B,
        'E': None,
        'tau': None,
        'theta': None,
        'E_tau_theta_results': None,
        'ccm_value': None,
        'convergence_p_value': None,
        'correlation': None
    }
    
    E_tau_theta_results = pd.DataFrame(columns = ['E', 'tau', 'theta', 'rho'])
    for E, tau, theta in list(product(E_list, tau_list, theta_list)):
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        try:
            c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        except:
            print(f"{interaction} did not converge")
            return default_output
        c = c['predictions'][['Observations', 'Predictions']]
        rho = c.corr().iloc[0,1]
        E_tau_theta_results.loc[len(E_tau_theta_results)] = [E, tau, theta, rho]
    E_tau_theta_results = E_tau_theta_results.apply(pd.to_numeric, errors='coerce')

    # Assign E, tau, and theta to be the optimal E, tau, and theta
    ccm_value = E_tau_theta_results['rho'].max()
    E = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'E'].item())
    tau = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'tau'].item())
    theta = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'theta'].item())
        
    # Get convergence p-value
    try:
        convergence_p_value = get_convergence_p_value(block, sample, A, B, E, Tp, tau, theta)
    except:
        print(f"{interaction} did not converge")
        return default_output

    # Preparing Output
    output = {
        'target (driver)': A,
        'lib (driven)': B,
        'E': E,
        'tau': tau,
        'theta': theta,
        'E_tau_theta_results': E_tau_theta_results,
        'ccm_value': ccm_value,
        'convergence_p_value': convergence_p_value,
        'correlation': df.corr().iloc[0,1]
    }

    return output

def get_convergence_p_value(df, sample, A, B, E, Tp, tau, theta):
    # Get convergence p-value for CCM (one-tailed t-test on cross-map values using 20% and 50% library sizes)
    # H0: μ_20% ≥ μ_50%
    # HA: μ_20% < μ_50%
    # If p < 0.05, the 20% library size trials have a rho that is significantly smaller than the 50% library trials  
    
    libsize1 = int(np.ceil(df.shape[0]/5))   # 20% of the full library size
    libsize2 = int(np.ceil(df.shape[0]/2))   # 50% of the full library size
    
    max_iterations = 10 * sample
    
    # Get list of rhos for libsize1
    rhos1 = []; iteration_count = 0
    while len(rhos1) < sample and iteration_count < max_iterations:
        start = np.random.randint(libsize1, len(df))
        library = [start - libsize1, start]
        data_subset = df.iloc[library[0]:library[1]]
        lib = f'{library[0]+1} {library[1]+1}'
        driver = f'{A}(t-0)'
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        c = c['predictions'][['Observations', 'Predictions']]
        rho1 = c.corr().iloc[0,1]
        if not np.isnan(rho1):
            rhos1.append(rho1)
        iteration_count += 1
        
    # Get list of rhos for libsize2
    rhos2 = []; iteration_count = 0
    while len(rhos2) < sample and iteration_count < max_iterations:
        start = np.random.randint(libsize2, len(df))
        library = [start - libsize2, start]
        data_subset = df.iloc[library[0]:library[1]]
        lib = f'{library[0]+1} {library[1]+1}'
        driver = f'{A}(t-0)'
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        c = c['predictions'][['Observations', 'Predictions']]
        rho2 = c.corr().iloc[0,1]
        if not np.isnan(rho2):
            rhos2.append(rho2)
        iteration_count += 1
    
    convergence_t_stat, convergence_p_value = ttest_ind(rhos1, rhos2, alternative='less')
    
    return convergence_p_value


In [18]:
HAB_data = pd.read_csv('Data/carter_data_w_gaps.csv', index_col=0) #LOAD DATA

Index(['Temp (deg C)', 'Chl1 (mg/m3)', 'Chl2 (mg/m3)', 'Avg Chloro (mg/m3)',
       'Phaeo1 (mg/m3)', 'Phaeo2 (mg/m3)', 'Avg Phaeo (mg/m3)', 'Nitrate (uM)',
       'Phosphate (uM)', 'Silicate (uM) ', 'Nitrite (uM)', 'Ammonium (uM)',
       ''CellCountDetection_Limit'', ''Total_Diatoms'', ''Akashiwo_sanguinea'',
       'Total_Tripos'', 'Total_Cochlodinium_spp'', ''Lingulodinium_polyedra'',
       'Total_Prorocentrum_spp'', ''Total_Dinoflagellates'',
       ''Total_Phytoplankton'', 'SURF_SAL_PSU', 'BOT_SAL_PSU', 'SURF_TEMP_C',
       'BOT_TEMP_C', 'WSPD', 'time'],
      dtype='object')

In [19]:
#HAB_data = pd.read_csv('Data/data_w_gaps_and_wind.csv', index_col=0)#.iloc[304:612] RANGE w/o missing values
HAB_data = HAB_data.reset_index()
HAB_data.columns = HAB_data.columns.str.replace(' ', '_')

# Put columns in alphabetical order
sorted_columns = sorted(HAB_data.columns)
HAB_data = HAB_data[sorted_columns]
HAB_data = HAB_data.set_index('time')
target = 'Avg_Chloro_(mg/m3)' #change target if needed

# Make indices integers and save mapping to dates
#date_to_int_map = {i: date for i, date in enumerate(HAB_data.index)}
#HAB_data.index = range(len(HAB_data))

HAB_data.columns

Index([''Akashiwo_sanguinea'', ''CellCountDetection_Limit'',
       ''Lingulodinium_polyedra'', ''Total_Diatoms'',
       ''Total_Dinoflagellates'', ''Total_Phytoplankton'', 'Ammonium_(uM)',
       'Avg_Chloro_(mg/m3)', 'Avg_Phaeo_(mg/m3)', 'BOT_SAL_PSU', 'BOT_TEMP_C',
       'Chl1_(mg/m3)', 'Chl2_(mg/m3)', 'Nitrate_(uM)', 'Nitrite_(uM)',
       'Phaeo1_(mg/m3)', 'Phaeo2_(mg/m3)', 'Phosphate_(uM)', 'SURF_SAL_PSU',
       'SURF_TEMP_C', 'Silicate_(uM)_', 'Temp_(deg_C)',
       'Total_Cochlodinium_spp'', 'Total_Prorocentrum_spp'', 'Total_Tripos'',
       'WSPD', 'index'],
      dtype='object')

In [6]:
print(HAB_data.isna().sum())

'Akashiwo_sanguinea'          768
'CellCountDetection_Limit'    768
'Lingulodinium_polyedra'      768
'Total_Diatoms'               768
'Total_Dinoflagellates'       768
'Total_Phytoplankton'         768
Ammonium_(uM)                 100
Avg_Chloro_(mg/m3)              2
Avg_Phaeo_(mg/m3)               2
BOT_SAL_PSU                   187
BOT_TEMP_C                    192
Chl1_(mg/m3)                    3
Chl2_(mg/m3)                    3
Nitrate_(uM)                   99
Nitrite_(uM)                   99
Phaeo1_(mg/m3)                  3
Phaeo2_(mg/m3)                  3
Phosphate_(uM)                 99
SURF_SAL_PSU                  105
SURF_TEMP_C                   110
Silicate_(uM)_                 99
Temp_(deg_C)                    0
Total_Cochlodinium_spp'       768
Total_Prorocentrum_spp'       768
Total_Tripos'                 906
WSPD                            0
index                           0
dtype: int64


In [7]:
#IMPUTE HAB DATA
#Build basic linear regression model as sanity check
# Custom impute missing values with the average of the value in front and behind of it 
class ForwardBackwardImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
        X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')

        return (X_filled_forward + X_filled_backward) / 2


Imputer = ForwardBackwardImputer()
HAB_data = HAB_data.apply(pd.to_numeric, errors='coerce')
Imputer.fit(HAB_data)
HAB_data = Imputer.transform(HAB_data)#COMMENT OUT IF DONT WANT MEAN MPUTE
HAB_data

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_59645/137097351.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_59645/137097351.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')


,'Akashiwo_sanguinea','CellCountDetection_Limit','Lingulodinium_polyedra','Total_Diatoms','Total_Dinoflagellates','Total_Phytoplankton',Ammonium_(uM),Avg_Chloro_(mg/m3),Avg_Phaeo_(mg/m3),BOT_SAL_PSU,...,Phosphate_(uM),SURF_SAL_PSU,SURF_TEMP_C,Silicate_(uM)_,Temp_(deg_C),Total_Cochlodinium_spp',Total_Prorocentrum_spp',Total_Tripos',WSPD,index
time,,,,,,,,,,,,,,,,,,,,,
0,0.0,89.0,89.0,69952.0,15684.0,85636.0,0.530,1.82,0.60,33.62,...,0.240,33.77,20.3,5.60,19.8,0.0,5168.0,0.0,4.691667,0.0
1,44.5,89.0,89.0,118740.5,10693.5,129434.5,0.795,2.08,0.53,33.60,...,0.305,33.74,19.9,5.35,19.7,0.0,3163.0,0.0,3.954167,1.0
2,89.0,89.0,89.0,167529.0,5703.0,173233.0,1.060,2.71,1.18,33.62,...,0.370,33.66,17.8,5.10,17.3,0.0,1158.0,0.0,2.787500,2.0
3,89.0,89.0,178.0,117315.0,8109.0,125424.5,0.705,1.32,0.52,33.53,...,0.305,33.44,19.8,4.30,19.5,133.5,1113.5,0.0,2.825000,3.0
4,89.0,89.0,267.0,67101.0,10515.0,77616.0,0.350,0.99,0.55,33.58,...,0.240,33.74,22.2,3.50,21.7,267.0,1069.0,0.0,3.412500,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,0.0,626.0,0.0,95122.0,63206.0,158328.0,0.410,3.38,1.00,33.36,...,0.200,33.38,13.5,4.10,13.8,0.0,1252.0,26910.0,5.088889,1597.0
1598,0.0,626.0,5006.5,191182.5,114209.0,305391.5,0.410,6.75,0.88,33.35,...,0.200,33.37,14.3,4.10,14.2,0.0,11890.5,51003.0,2.736806,1598.0
1599,0.0,626.0,10013.0,287243.0,165212.0,452455.0,0.410,9.71,1.49,33.36,...,0.200,33.36,13.3,4.10,13.2,0.0,22529.0,75096.0,2.813194,1599.0


In [8]:
print(HAB_data.isna().sum())

'Akashiwo_sanguinea'          0
'CellCountDetection_Limit'    0
'Lingulodinium_polyedra'      0
'Total_Diatoms'               0
'Total_Dinoflagellates'       0
'Total_Phytoplankton'         0
Ammonium_(uM)                 0
Avg_Chloro_(mg/m3)            0
Avg_Phaeo_(mg/m3)             0
BOT_SAL_PSU                   0
BOT_TEMP_C                    0
Chl1_(mg/m3)                  0
Chl2_(mg/m3)                  0
Nitrate_(uM)                  0
Nitrite_(uM)                  0
Phaeo1_(mg/m3)                0
Phaeo2_(mg/m3)                0
Phosphate_(uM)                0
SURF_SAL_PSU                  0
SURF_TEMP_C                   0
Silicate_(uM)_                0
Temp_(deg_C)                  0
Total_Cochlodinium_spp'       0
Total_Prorocentrum_spp'       0
Total_Tripos'                 0
WSPD                          0
index                         0
dtype: int64


In [9]:
#STANDARDIZE
#LEFT OFF HERE THERE IS AN ERROR
'''
scaler = MinMaxScaler()
columns = ['AVG_DENS_kgm3', 'AVG_SAL_PSU', 'AVG_TEMP_C', 'Avg_Chloro', 
           'BOT_DENS_kgm3', 'BOT_SAL_PSU', 'BOT_TEMP_C', 'Nitrate', 'Nitrite', 
           'Phosphate', 'SURF_DENS_kgm3', 'SURF_SAL_PSU', 'SURF_TEMP_C', 
           'Silicate', 'WSPD']
HAB_data[columns] = scaler.fit_transform(HAB_data[columns])
HAB_data
'''

"\nscaler = MinMaxScaler()\ncolumns = ['AVG_DENS_kgm3', 'AVG_SAL_PSU', 'AVG_TEMP_C', 'Avg_Chloro', \n           'BOT_DENS_kgm3', 'BOT_SAL_PSU', 'BOT_TEMP_C', 'Nitrate', 'Nitrite', \n           'Phosphate', 'SURF_DENS_kgm3', 'SURF_SAL_PSU', 'SURF_TEMP_C', \n           'Silicate', 'WSPD']\nHAB_data[columns] = scaler.fit_transform(HAB_data[columns])\nHAB_data\n"

In [10]:
#HAB_data.drop(columns=[]) # FOR MELISSA CARTER DATA
block = get_block(HAB_data, num_lags=50, tau=1)
block

,'Akashiwo_sanguinea'(t-0),'CellCountDetection_Limit'(t-0),'Lingulodinium_polyedra'(t-0),'Total_Diatoms'(t-0),'Total_Dinoflagellates'(t-0),'Total_Phytoplankton'(t-0),Ammonium_(uM)(t-0),Avg_Chloro_(mg/m3)(t-0),Avg_Phaeo_(mg/m3)(t-0),BOT_SAL_PSU(t-0),...,Phosphate_(uM)(t-50),SURF_SAL_PSU(t-50),SURF_TEMP_C(t-50),Silicate_(uM)_(t-50),Temp_(deg_C)(t-50),Total_Cochlodinium_spp'(t-50),Total_Prorocentrum_spp'(t-50),Total_Tripos'(t-50),WSPD(t-50),index(t-50)
time,,,,,,,,,,,,,,,,,,,,,
0,0.0,89.0,89.0,69952.0,15684.0,85636.0,0.530,1.82,0.60,33.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44.5,89.0,89.0,118740.5,10693.5,129434.5,0.795,2.08,0.53,33.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,89.0,89.0,89.0,167529.0,5703.0,173233.0,1.060,2.71,1.18,33.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,89.0,89.0,178.0,117315.0,8109.0,125424.5,0.705,1.32,0.52,33.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,89.0,89.0,267.0,67101.0,10515.0,77616.0,0.350,0.99,0.55,33.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,0.0,626.0,0.0,95122.0,63206.0,158328.0,0.410,3.38,1.00,33.36,...,0.22,33.37,17.9,8.1,17.5,9387.0,64458.0,0.0,1.965278,1547.0
1598,0.0,626.0,5006.5,191182.5,114209.0,305391.5,0.410,6.75,0.88,33.35,...,0.30,33.48,17.4,6.2,16.7,3755.0,28787.0,1252.0,5.076389,1548.0
1599,0.0,626.0,10013.0,287243.0,165212.0,452455.0,0.410,9.71,1.49,33.36,...,0.19,33.52,18.5,4.2,17.6,3755.0,26909.5,626.0,3.731250,1549.0


In [11]:
E_list = range(2,13)
tau_list = [-1,-2,-3] #can try more taus
theta_list = [0,0.1,0.5,1,2,3,4,5,6,7,8,9]
Tp = 0
exclusion_radius = 0

all_ccm_results = pd.DataFrame()
interactions = list(permutations(HAB_data.columns.tolist(),2))
print(interactions)
target_interactions = [pair for pair in interactions if target in pair]

interaction = target_interactions[0]
print(f'There are {len(target_interactions)} interactions')

results = Parallel(n_jobs=4)(
    delayed(ccm)(interaction, block, E_list, tau_list, theta_list, Tp) for interaction in target_interactions)
results_df = pd.DataFrame(results)

[("'Akashiwo_sanguinea'", "'CellCountDetection_Limit'"), ("'Akashiwo_sanguinea'", "'Lingulodinium_polyedra'"), ("'Akashiwo_sanguinea'", "'Total_Diatoms'"), ("'Akashiwo_sanguinea'", "'Total_Dinoflagellates'"), ("'Akashiwo_sanguinea'", "'Total_Phytoplankton'"), ("'Akashiwo_sanguinea'", 'Ammonium_(uM)'), ("'Akashiwo_sanguinea'", 'Avg_Chloro_(mg/m3)'), ("'Akashiwo_sanguinea'", 'Avg_Phaeo_(mg/m3)'), ("'Akashiwo_sanguinea'", 'BOT_SAL_PSU'), ("'Akashiwo_sanguinea'", 'BOT_TEMP_C'), ("'Akashiwo_sanguinea'", 'Chl1_(mg/m3)'), ("'Akashiwo_sanguinea'", 'Chl2_(mg/m3)'), ("'Akashiwo_sanguinea'", 'Nitrate_(uM)'), ("'Akashiwo_sanguinea'", 'Nitrite_(uM)'), ("'Akashiwo_sanguinea'", 'Phaeo1_(mg/m3)'), ("'Akashiwo_sanguinea'", 'Phaeo2_(mg/m3)'), ("'Akashiwo_sanguinea'", 'Phosphate_(uM)'), ("'Akashiwo_sanguinea'", 'SURF_SAL_PSU'), ("'Akashiwo_sanguinea'", 'SURF_TEMP_C'), ("'Akashiwo_sanguinea'", 'Silicate_(uM)_'), ("'Akashiwo_sanguinea'", 'Temp_(deg_C)'), ("'Akashiwo_sanguinea'", "Total_Cochlodinium_spp'"),

("'Akashiwo_sanguinea'", 'Avg_Chloro_(mg/m3)')
("'CellCountDetection_Limit'", 'Avg_Chloro_(mg/m3)')
("'Lingulodinium_polyedra'", 'Avg_Chloro_(mg/m3)')
("'Total_Diatoms'", 'Avg_Chloro_(mg/m3)')
("'Total_Dinoflagellates'", 'Avg_Chloro_(mg/m3)')
("'Total_Phytoplankton'", 'Avg_Chloro_(mg/m3)')
('Ammonium_(uM)', 'Avg_Chloro_(mg/m3)')
('Avg_Chloro_(mg/m3)', "'Akashiwo_sanguinea'")
('Avg_Chloro_(mg/m3)', "'CellCountDetection_Limit'")
('Avg_Chloro_(mg/m3)', "'Lingulodinium_polyedra'")
('Avg_Chloro_(mg/m3)', "'Total_Diatoms'")
('Avg_Chloro_(mg/m3)', "'Total_Dinoflagellates'")
('Avg_Chloro_(mg/m3)', "'Total_Phytoplankton'")
('Avg_Chloro_(mg/m3)', 'Ammonium_(uM)')
('Avg_Chloro_(mg/m3)', 'Avg_Phaeo_(mg/m3)')
('Avg_Chloro_(mg/m3)', 'BOT_SAL_PSU')
('Avg_Chloro_(mg/m3)', 'BOT_TEMP_C')
('Avg_Chloro_(mg/m3)', 'Chl1_(mg/m3)')
('Avg_Chloro_(mg/m3)', 'Chl2_(mg/m3)')
('Avg_Chloro_(mg/m3)', 'Nitrate_(uM)')
('Avg_Chloro_(mg/m3)', 'Nitrite_(uM)')
('Avg_Chloro_(mg/m3)', 'Phaeo1_(mg/m3)')
('Avg_Chloro_(mg/m3)',

/opt/miniconda3/envs/pyedm_env/lib/python3.9/site-packages/pyEDM/SMap.py:119: RuntimeWarning: divide by zero encountered in divide
  distRowScale = self.theta / distRowMean
/opt/miniconda3/envs/pyedm_env/lib/python3.9/site-packages/pyEDM/SMap.py:120: RuntimeWarning: invalid value encountered in multiply
  W = exp( -distRowScale[:,None] * self.knn_distances )


 ** On entry to DLASCL parameter number  4 had an illegal value
 ** On entry to DLASCL parameter number  4 had an illegal value
('Avg_Chloro_(mg/m3)', 'Nitrite_(uM)') did not converge
('Avg_Chloro_(mg/m3)', 'SURF_SAL_PSU')
('Avg_Chloro_(mg/m3)', 'SURF_TEMP_C')
('Avg_Chloro_(mg/m3)', 'Silicate_(uM)_')
('Avg_Chloro_(mg/m3)', 'Temp_(deg_C)')
('Avg_Chloro_(mg/m3)', "Total_Cochlodinium_spp'")
('Avg_Chloro_(mg/m3)', "Total_Prorocentrum_spp'")
('Avg_Chloro_(mg/m3)', "Total_Tripos'")
('Avg_Chloro_(mg/m3)', 'WSPD')
('Avg_Chloro_(mg/m3)', 'index')
('Avg_Phaeo_(mg/m3)', 'Avg_Chloro_(mg/m3)')
('BOT_SAL_PSU', 'Avg_Chloro_(mg/m3)')
('BOT_TEMP_C', 'Avg_Chloro_(mg/m3)')
('Chl1_(mg/m3)', 'Avg_Chloro_(mg/m3)')
('Chl2_(mg/m3)', 'Avg_Chloro_(mg/m3)')
('Nitrate_(uM)', 'Avg_Chloro_(mg/m3)')
('Nitrite_(uM)', 'Avg_Chloro_(mg/m3)')
('Phaeo1_(mg/m3)', 'Avg_Chloro_(mg/m3)')
('Phaeo2_(mg/m3)', 'Avg_Chloro_(mg/m3)')
('Phosphate_(uM)', 'Avg_Chloro_(mg/m3)')
('SURF_SAL_PSU', 'Avg_Chloro_(mg/m3)')
('SURF_TEMP_C', 'Av

In [16]:
# Get CCM results that show convergence (convergence p-value < 0.05)
results_df = pd.DataFrame(results)
ccm_cutoff = -1

significant_results = results_df[results_df.convergence_p_value<0.20]
significant_results = significant_results.sort_values(by='ccm_value', ascending=False)
significant_results = significant_results[['target (driver)', 'lib (driven)', 'E', 'tau', 'theta', 'ccm_value']].reset_index(drop=True)

display(significant_results[significant_results.ccm_value>ccm_cutoff])

# Choose system variables where the CCM value to or from the target is > ccm_cutoff
system_variables = significant_results[significant_results.ccm_value > ccm_cutoff]
system_variables = system_variables[['target (driver)', 'lib (driven)']].values.flatten().tolist()
system_variables = list(set(system_variables))
print('system variables: ')
display(sorted(system_variables))

,target (driver),lib (driven),E,tau,theta,ccm_value
0,Chl1_(mg/m3),Avg_Chloro_(mg/m3),2.0,-2.0,2.0,0.996579
1,'Lingulodinium_polyedra',Avg_Chloro_(mg/m3),2.0,-1.0,0.0,0.969650
2,Avg_Chloro_(mg/m3),'Lingulodinium_polyedra',2.0,-1.0,8.0,0.964027
3,'Total_Dinoflagellates',Avg_Chloro_(mg/m3),2.0,-1.0,0.0,0.938090
4,Avg_Chloro_(mg/m3),'Total_Dinoflagellates',2.0,-1.0,2.0,0.933684
5,Avg_Chloro_(mg/m3),'CellCountDetection_Limit',5.0,-3.0,6.0,0.922070
6,Phaeo2_(mg/m3),Avg_Chloro_(mg/m3),2.0,-3.0,0.0,0.903395
7,Avg_Chloro_(mg/m3),'Total_Phytoplankton',2.0,-3.0,8.0,0.902716
8,'CellCountDetection_Limit',Avg_Chloro_(mg/m3),5.0,-1.0,3.0,0.879920
9,'Total_Phytoplankton',Avg_Chloro_(mg/m3),9.0,-1.0,0.0,0.879064


system variables: 


["'Akashiwo_sanguinea'",
 "'CellCountDetection_Limit'",
 "'Lingulodinium_polyedra'",
 "'Total_Diatoms'",
 "'Total_Dinoflagellates'",
 "'Total_Phytoplankton'",
 'Ammonium_(uM)',
 'Avg_Chloro_(mg/m3)',
 'Avg_Phaeo_(mg/m3)',
 'BOT_SAL_PSU',
 'BOT_TEMP_C',
 'Chl1_(mg/m3)',
 'Nitrate_(uM)',
 'Phaeo1_(mg/m3)',
 'Phaeo2_(mg/m3)',
 "Total_Cochlodinium_spp'",
 "Total_Prorocentrum_spp'"]

In [13]:
results_df[results_df.convergence_p_value<0.8].sort_values(by='convergence_p_value', ascending=False)

,target (driver),lib (driven),E,tau,theta,E_tau_theta_results,ccm_value,convergence_p_value,correlation
18,Avg_Chloro_(mg/m3),Chl2_(mg/m3),2.0,-2.0,2.0,E tau theta rho 0 2.0 -1.0...,0.997193,7.055352e-01,0.997727
42,Phosphate_(uM),Avg_Chloro_(mg/m3),5.0,-1.0,0.0,E tau theta rho 0 2.0 -1.0...,0.083281,5.455458e-01,0.168929
50,WSPD,Avg_Chloro_(mg/m3),7.0,-3.0,9.0,E tau theta rho 0 2.0 -1.0...,0.128322,3.502907e-01,-0.017667
31,Avg_Chloro_(mg/m3),WSPD,8.0,-2.0,3.0,E tau theta rho 0 2.0 -1.0...,0.059044,3.389087e-01,-0.017667
24,Avg_Chloro_(mg/m3),SURF_SAL_PSU,12.0,-3.0,9.0,E tau theta rho 0 2.0 -1.0...,0.527145,2.640243e-01,-0.005291
16,Avg_Chloro_(mg/m3),BOT_TEMP_C,2.0,-2.0,9.0,E tau theta rho 0 2.0 -1.0...,0.273628,1.833696e-01,-0.023606
36,Chl1_(mg/m3),Avg_Chloro_(mg/m3),2.0,-2.0,2.0,E tau theta rho 0 2.0 -1.0...,0.996579,1.731781e-01,0.997916
22,Avg_Chloro_(mg/m3),Phaeo2_(mg/m3),3.0,-2.0,9.0,E tau theta rho 0 2.0 -1.0...,0.805851,1.572402e-01,0.916862
15,Avg_Chloro_(mg/m3),BOT_SAL_PSU,11.0,-3.0,9.0,E tau theta rho 0 2.0 -1.0...,0.792634,1.442382e-01,0.035468
29,Avg_Chloro_(mg/m3),Total_Prorocentrum_spp',4.0,-1.0,9.0,E tau theta rho 0 2.0 -1.0...,0.671596,1.133014e-01,0.078730
